In [1]:
import numpy as np 
import pandas as pd
import time
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import re
# import math
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# from lightgbm import LGBMRegressor, LGBMClassifier
# from xgboost import XGBRegressor, XGBClassifier
# from catboost import CatBoostRegressor, CatBoostClassifier
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cab
from sklearn import metrics
# from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
# from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
# from sklearn.linear_model import LogisticRegression, SGDClassifier
# from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
# from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
# from sklearn import metrics
# from sklearn.svm import SVC
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
# from collections import defaultdict, Counter
import warnings
import json 
import pickle
warnings.filterwarnings('ignore')

import pickle
import random
from tqdm import tqdm
# import sentence_transformers 
# from sklearn.preprocessing import KBinsDiscretizer
# from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from gensim.models import Word2Vec

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Dataset

from utils import set_random_seed, get_logger, ensure_dir, save_model_with_epoch, load_model_with_epoch
from data import NNDataset, NNDatasetV3
from model import MatchModel, BaseModel, MatchModelV2, BinaryModel

"""
相比于trainDeeep.py，加入一些手动聚合的序列特征，例如历史序列的平均价格，历史序列的不同类别数之类的
SeqFeatureEmbedding现在使用2层全连接，可以换多层！
"""

emb_dim = 16
dense_bins = 10
hid_dim = 256
dropout = 0.3
layers = 4
bidirectional = False
seq_emb_factor = 4  # 人工序列特征的嵌入是emb_dim的几倍

batch_size = 1024
epochs = 100
len_candidate_set = 10
Fold = 0
device = torch.device('cuda:2')

train = True
load_init = False
load_epoch = 28
load_exp_id = 11088

# TODO: 调整batch-size， 调整hidden-size
# TODO: 跑Fold 4
learning_rate = 0.001
weight_decay = 0.00001
early_stop_lr = 1e-6
lr_patience = 5
lr_decay_ratio = 0.1
clip = 5
log_every = 100
early_stop = True
patience = 10
kfold = 5
attn_match = True 

w2v_window = 3
w2v_min_count = 1
w2v_epochs = 500
w2v_vector_size = 128

seed = 2023
set_random_seed(seed)

model_name = 'BinaryModelFold{}'.format(Fold)
loc2id = {'DE': 0, 'JP': 1, 'UK': 2, 'ES': 3, 'FR': 4, 'IT': 5}

config = locals()

dense_norm = True

In [2]:
# 加载必要的数据

exp_id = config.get('exp_id', None)
if exp_id is None:
    exp_id = int(random.SystemRandom().random() * 100000)
    config['exp_id'] = exp_id

logger = get_logger(config)
logger.info('Exp_id {}'.format(exp_id))
logger.info(config)

logger.info('read data')

titles_embedding = np.load('./data/titles_embedding.npy')
descs_embedding = np.load('./data/descs_embedding.npy')
logger.info('titles_embedding: {}'.format(titles_embedding.shape))
logger.info('descs_embedding: {}'.format(descs_embedding.shape))

product2id = json.load(open('data/product2id.json', 'r'))
id2product = json.load(open('data/id2product.json', 'r'))
id2product = {int(k): v for k, v in id2product.items()}
logger.info('product2id: {}'.format(len(product2id)))
logger.info('id2product: {}'.format(len(id2product)))

word2vec_embedding = np.load('./data/word2vec_embedding.npy')
logger.info('word2vec_embedding: {}'.format(word2vec_embedding.shape))

# top200 = pickle.load(open('data/top200_new.pkl', 'rb'))

df_train_encoded = pd.read_csv('data/df_train_encoded.csv')
df_test_encoded = pd.read_csv('data/df_test_encoded_phase2.csv')
products_encoded = pd.read_csv('./data/products_encoded.csv')
logger.info('df_train_encoded: {}'.format(df_train_encoded.shape))
logger.info('df_test_encoded: {}'.format(df_test_encoded.shape))
logger.info('products_encoded: {}'.format(products_encoded.shape))

2023-05-24 14:43:58,769 - INFO - Log directory: ./log
2023-05-24 14:43:58,770 - INFO - Exp_id 99521
2023-05-24 14:43:58,770 - INFO - {'__name__': '__main__', '__doc__': '\n相比于trainDeeep.py，加入一些手动聚合的序列特征，例如历史序列的平均价格，历史序列的不同类别数之类的\nSeqFeatureEmbedding现在使用2层全连接，可以换多层！\n', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'import numpy as np \nimport pandas as pd\nimport time\n# import matplotlib.pyplot as plt\n# import seaborn as sns\n# import os\n# import re\n# import math\n# import plotly.express as px\n# import plotly.graph_objects as go\n# from plotly.subplots import make_subplots\n\n# from lightgbm import LGBMRegressor, LGBMClassifier\n# from xgboost import XGBRegressor, XGBClassifier\n# from catboost import CatBoostRegressor, CatBoostClassifier\n# import lightgbm as lgb\n# import xgboost as xgb\n# import catboost as cab\nfrom sklearn import metrics\n# from sklearn.prep

In [3]:
num_features = ['price', 'len_title', 'len_desc']
if dense_norm:
    logger.info('MinMaxScaler Norm products_num_feas')
    mms = MinMaxScaler(feature_range=(0,1))
    products_encoded[num_features] = mms.fit_transform(products_encoded[num_features])
for fe in num_features:
    products_encoded[fe] = products_encoded[fe].astype('float32')
    assert products_encoded[fe].dtypes == 'float32'

id_count = products_encoded.shape[0]

train_preds_encoded = pickle.load(open('./data/train_preds_all_encoded_new_phase2.pkl', 'rb'))  # (len_train, 100)
test_preds_encoded = pickle.load(open('./data/test_preds_all_encoded_new_phase2.pkl', 'rb'))  # (len_test, 100)
test_preds = pickle.load(open('./data/test_preds_phase2.pkl', 'rb'))
logger.info('train_preds_encoded: {}'.format(len(train_preds_encoded)))
logger.info('test_preds_encoded: {}'.format(len(test_preds_encoded)))
logger.info('test_preds: {}'.format(len(test_preds)))

logger.info('Cutting the candidate_set to {}'.format(len_candidate_set))
# TODO: 可以改成保障一个正样本，补充9个负样本
cut_train_preds_encoded = [lst[:len_candidate_set] for lst in tqdm(train_preds_encoded, total=len(train_preds_encoded))]
df_train_encoded['recall'] = cut_train_preds_encoded
cut_test_preds_encoded = [lst[:len_candidate_set] for lst in tqdm(test_preds_encoded, total=len(test_preds_encoded))]
df_test_encoded['recall'] = cut_test_preds_encoded

logger.info('Eval the prev_items')
df_train_encoded['prev_items'] = df_train_encoded['prev_items'].apply(eval)
df_test_encoded['prev_items'] = df_test_encoded['prev_items'].apply(eval)

2023-05-24 14:45:19,655 - INFO - MinMaxScaler Norm products_num_feas
2023-05-24 14:46:13,571 - INFO - train_preds_encoded: 3606249
2023-05-24 14:46:13,572 - INFO - test_preds_encoded: 316972
2023-05-24 14:46:13,573 - INFO - test_preds: 316972
2023-05-24 14:46:13,574 - INFO - Cutting the candidate_set to 10


100%|██████████| 316972/316972 [00:08<00:00, 38382.50it/s]  

2023-05-24 14:46:39,435 - INFO - Eval the prev_items


In [4]:
logger.info('Load Hand-made Seq Features')
df_train_seqs_feas_all = pd.read_csv('data/df_train_seqs_feas_all.csv')  # 29维特征
df_test_seqs_feas_all = pd.read_csv('data/df_test_seqs_feas_all_phase2.csv')
logger.info('df_train_seqs_feas_all: {}'.format(df_train_seqs_feas_all.shape))
logger.info('df_test_seqs_feas_all: {}'.format(df_test_seqs_feas_all.shape))
seqs_cat_feas = [f for f in df_train_seqs_feas_all.columns if 'NUNIQUE' in f or 'COUNT' in f]
seqs_num_feas = [f for f in df_train_seqs_feas_all.columns if f not in seqs_cat_feas]
logger.info('seqs_cat_feas: {}'.format(seqs_cat_feas))
logger.info('seqs_num_feas: {}'.format(seqs_num_feas))

if dense_norm:
    logger.info('MinMaxScaler Norm seqs_num_feas')
    mms = MinMaxScaler(feature_range=(0,1))
    df_train_seqs_feas_all[seqs_num_feas] = mms.fit_transform(df_train_seqs_feas_all[seqs_num_feas])
    df_test_seqs_feas_all[seqs_num_feas] = mms.fit_transform(df_test_seqs_feas_all[seqs_num_feas])

for fe in seqs_num_feas:
    df_train_seqs_feas_all[fe] = df_train_seqs_feas_all[fe].astype('float32')
    df_test_seqs_feas_all[fe] = df_test_seqs_feas_all[fe].astype('float32')

df_train_all = pd.concat([df_train_encoded, df_train_seqs_feas_all], axis=1)
df_test_all = pd.concat([df_test_encoded, df_test_seqs_feas_all], axis=1)
logger.info('df_train_all: {}'.format(df_train_all.shape))
logger.info('df_test_all: {}'.format(df_test_all.shape))

df_train_all_exploded = df_train_all.explode('recall')
df_test_all_exploded = df_test_all.explode('recall')

df_train_all_exploded['label'] = df_train_all_exploded['next_item'] == df_train_all_exploded['recall']
df_test_all_exploded['label'] = df_test_all_exploded['next_item'] == df_test_all_exploded['recall']

df_train_all_exploded['index'] = df_train_all_exploded.index 
df_test_all_exploded['index'] = df_test_all_exploded.index 

logger.info('df_train_all_exploded: {}'.format(df_train_all_exploded.shape))
logger.info('df_test_all_exploded: {}'.format(df_test_all_exploded.shape))

logger.info('df_train_all_exploded.label.sum: {}'.format(df_train_all_exploded['label'].sum()))
logger.info('df_test_all_exploded.label.sum: {}'.format(df_test_all_exploded['label'].sum()))

df_train_encoded_exploded = df_train_all_exploded[['prev_items', 'next_item', 'locale', 'recall', 'label', 'index']]
df_train_seqs_cat_feas = df_train_all_exploded[seqs_cat_feas]
df_train_seqs_num_feas = df_train_all_exploded[seqs_num_feas]
df_test_encoded_exploded = df_test_all_exploded[['prev_items', 'next_item', 'locale', 'recall', 'label', 'index']]
df_test_seqs_cat_feas = df_test_all_exploded[seqs_cat_feas]
df_test_seqs_num_feas = df_test_all_exploded[seqs_num_feas]

del df_train_all_exploded
del df_test_all_exploded
del df_train_all
del df_test_all

logger.info('df_train_encoded_exploded: {}'.format(df_train_encoded_exploded.shape))
logger.info('df_train_seqs_cat_feas: {}'.format(df_train_seqs_cat_feas.shape))
logger.info('df_train_seqs_num_feas: {}'.format(df_train_seqs_num_feas.shape))
logger.info('df_test_encoded_exploded: {}'.format(df_test_encoded_exploded.shape))
logger.info('df_test_seqs_cat_feas: {}'.format(df_test_seqs_cat_feas.shape))
logger.info('df_test_seqs_num_feas: {}'.format(df_test_seqs_num_feas.shape))

2023-05-24 14:47:28,376 - INFO - Load Hand-made Seq Features
2023-05-24 14:47:38,736 - INFO - df_train_seqs_feas_all: (3606249, 29)
2023-05-24 14:47:38,737 - INFO - df_test_seqs_feas_all: (316972, 29)
2023-05-24 14:47:38,738 - INFO - seqs_cat_feas: ['idNUNIQUE', 'idCOUNT', 'brandNUNIQUE', 'brandCOUNT', 'colorNUNIQUE', 'colorCOUNT', 'sizeNUNIQUE', 'sizeCOUNT', 'modelNUNIQUE', 'modelCOUNT', 'materialNUNIQUE', 'materialCOUNT', 'authorNUNIQUE', 'authorCOUNT']
2023-05-24 14:47:38,738 - INFO - seqs_num_feas: ['priceMEAN', 'priceSTD', 'priceMIN', 'priceMAX', 'priceSUM', 'len_titleMEAN', 'len_titleSTD', 'len_titleMIN', 'len_titleMAX', 'len_titleSUM', 'len_descMEAN', 'len_descSTD', 'len_descMIN', 'len_descMAX', 'len_descSUM']
2023-05-24 14:47:38,739 - INFO - MinMaxScaler Norm seqs_num_feas
2023-05-24 14:47:40,284 - INFO - df_train_all: (3606249, 34)
2023-05-24 14:47:40,285 - INFO - df_test_all: (316972, 34)
2023-05-24 14:48:02,765 - INFO - df_train_all_exploded: (36062490, 36)
2023-05-24 14:48:

In [5]:
df_test = pd.read_csv('data/phase2/sessions_test_task1.csv')
logger.info('df_test: {}'.format(df_test.shape))

tmp = pd.concat([df_train_seqs_feas_all[seqs_cat_feas], df_test_seqs_feas_all[seqs_cat_feas]])
tmp_nunique = (tmp.max() + 1).to_dict()  # 不是nunique，因为这个是计数特征，不是连续的0~n-1

data_feature = {}
data_feature['len_encode_brand'] = products_encoded['encode_brand'].nunique()
data_feature['len_encode_color'] = products_encoded['encode_color'].nunique()
data_feature['len_encode_size'] = products_encoded['encode_size'].nunique()
data_feature['len_encode_model'] = products_encoded['encode_model'].nunique()
data_feature['len_encode_material'] = products_encoded['encode_material'].nunique()
data_feature['len_encode_author'] = products_encoded['encode_author'].nunique()
data_feature['len_locale'] = len(loc2id)
data_feature['dense_bins'] = dense_bins
data_feature['id_count'] = id_count
data_feature['len_features'] = products_encoded.shape[1] - 1
data_feature['len_emb_features'] = 3
data_feature['len_candidate_set'] = len_candidate_set
data_feature['w2v_vector_size'] = w2v_vector_size
data_feature['sentence_vector_size'] = 384
data_feature['len_seqs_cat_feas'] = len(seqs_cat_feas)
data_feature['len_seqs_num_feas'] = len(seqs_num_feas)
data_feature['seq_emb_factor'] = seq_emb_factor
data_feature.update(tmp_nunique)
logger.info('data_feature:')
logger.info(data_feature)

del tmp

2023-05-24 14:48:07,651 - INFO - df_test: (316972, 2)
2023-05-24 14:48:08,067 - INFO - data_feature:
2023-05-24 14:48:08,068 - INFO - {'len_encode_brand': 177190, 'len_encode_color': 203261, 'len_encode_size': 218061, 'len_encode_model': 524102, 'len_encode_material': 45569, 'len_encode_author': 30836, 'len_locale': 6, 'dense_bins': 10, 'id_count': 1410675, 'len_features': 13, 'len_emb_features': 3, 'len_candidate_set': 10, 'w2v_vector_size': 128, 'sentence_vector_size': 384, 'len_seqs_cat_feas': 14, 'len_seqs_num_feas': 15, 'seq_emb_factor': 4, 'idNUNIQUE': 133, 'idCOUNT': 475, 'brandNUNIQUE': 39, 'brandCOUNT': 475, 'colorNUNIQUE': 54, 'colorCOUNT': 475, 'sizeNUNIQUE': 116, 'sizeCOUNT': 475, 'modelNUNIQUE': 59, 'modelCOUNT': 475, 'materialNUNIQUE': 26, 'materialCOUNT': 475, 'authorNUNIQUE': 65, 'authorCOUNT': 475}


In [6]:
fold_size = int(df_train_encoded_exploded.shape[0] / len_candidate_set // kfold) * len_candidate_set  # 7212490
index_list = list(range(0, len(df_train_encoded_exploded)))

In [7]:
val_indexes = []
start = 0
for i in range(kfold):
    end = min(start + fold_size, len(index_list))
    print(start, end)
    val_indexes.append(index_list[start:end])
    # print(np.min(val_indexes[-1]), np.max(val_indexes[-1]), len(val_indexes[-1]))
    start = end

0 7212490
7212490 14424980
14424980 21637470
21637470 28849960
28849960 36062450


In [8]:
train_indexes = []
for i, fold in enumerate(val_indexes):
    print(f"Fold {i+1}:")
    print("Validation set:", len(fold))
    train_indexes.append([index for sublist in val_indexes[:i] + val_indexes[i+1:] for index in sublist])
    print("Training set:", len(train_indexes[-1]))

Fold 1:
Validation set: 7212490
Training set: 28849960
Fold 2:
Validation set: 7212490
Training set: 28849960
Fold 3:
Validation set: 7212490
Training set: 28849960
Fold 4:
Validation set: 7212490
Training set: 28849960
Fold 5:
Validation set: 7212490
Training set: 28849960


In [9]:
np.save('data/5fold_trn_idx_list_binary_phase2_{}.npy'.format(len_candidate_set), train_indexes)
np.save('data/5fold_val_idx_list_binary_phase2_{}.npy'.format(len_candidate_set), val_indexes)

In [9]:
# 加载模型等

logger.info('create model')

products_input = {name: torch.tensor(products_encoded[name].values).to(device) for name in products_encoded.columns}

2023-05-24 14:49:45,930 - INFO - create model


In [11]:
config['heads'] = 8
config['seq_model'] = 'transformer'

In [12]:
model = BinaryModel(config, data_feature, products_input, word2vec_embedding, titles_embedding, descs_embedding).to(device)
logger.info(model)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=lr_patience, factor=lr_decay_ratio)

for name, param in model.named_parameters():
    logger.info(str(name) + '\t' + str(param.shape) + '\t' +
                              str(param.device) + '\t' + str(param.requires_grad))
total_num = sum([param.nelement() for param in model.parameters()])
logger.info('Total parameter numbers: {}'.format(total_num))


# 数据集DataLoader

trn_idx_list = np.load('data/5fold_trn_idx_list_binary_phase2_{}.npy'.format(len_candidate_set), allow_pickle=True)
val_idx_list = np.load('data/5fold_val_idx_list_binary_phase2_{}.npy'.format(len_candidate_set), allow_pickle=True)
logger.info('Fold {}: trn_idx {}'.format(Fold, len(trn_idx_list[Fold])))
logger.info('Fold {}: val_idx {}'.format(Fold, len(val_idx_list[Fold])))

KeyError: 'add_title'

In [32]:
train_set = NNDatasetV3(df_train_encoded_exploded.iloc[trn_idx_list[Fold]], 
                        df_train_seqs_cat_feas.iloc[trn_idx_list[Fold]], 
                        df_train_seqs_num_feas.iloc[trn_idx_list[Fold]])
val_set = NNDatasetV3(df_train_encoded_exploded.iloc[val_idx_list[Fold]], 
                      df_train_seqs_cat_feas.iloc[val_idx_list[Fold]], 
                      df_train_seqs_num_feas.iloc[val_idx_list[Fold]])
test_set = NNDatasetV3(df_test_encoded_exploded, df_test_seqs_cat_feas, 
                       df_test_seqs_num_feas)
logger.info('train_set: {}'.format(len(train_set)))
logger.info('val_set: {}'.format(len(val_set)))
logger.info('test_set: {}'.format(len(test_set)))

2023-05-24 05:38:55,366 - INFO - train_set: 28849960
2023-05-24 05:38:55,368 - INFO - val_set: 7212490
2023-05-24 05:38:55,369 - INFO - test_set: 3169720


In [33]:
def collate_fn(indices):
    batch_prev_items = []
    batch_locale = []
    batch_len = []
    batch_mask = []
    batch_candidate = []
    batch_origin_label = []
    batch_label = []
    batch_seq_cat = []
    batch_seq_num = []
    batch_index = []
    for item in indices:
        batch_len.append(len(item[0]))  # prev_items
    max_len = max(batch_len)
    for item in indices:
        l = len(item[0])
        batch_mask.append([1] * (l) + [0] * (max_len - l))  # 0代表padding的位置，需要mask
    for item in indices:
        # ['prev_items', 'locale', 'recall', 'next_item', label', 'index', 'seqs_cat_feas', 'seqs_num_feas']
        prev_items = item[0].copy()
        while (len(prev_items) < max_len):
            prev_items.append(id_count)  # embdding的时候id_count+1，把id_count作为padding了
        batch_prev_items.append(prev_items)
        batch_locale.append(item[1])
        batch_candidate.append(item[2])
        batch_origin_label.append(item[3])
        batch_label.append(item[4])
        batch_index.append(item[5])
        batch_seq_cat.append(item[6])
        batch_seq_num.append(item[7])
    return [torch.LongTensor(batch_prev_items).to(device), torch.LongTensor(batch_locale).to(device), 
            torch.LongTensor(batch_len).to(device), torch.LongTensor(batch_mask).to(device), 
            torch.LongTensor(batch_candidate).to(device), torch.FloatTensor(batch_label).to(device), 
            torch.LongTensor(batch_origin_label).to(device), torch.LongTensor(batch_index).to(device), 
            torch.LongTensor(batch_seq_cat).to(device), torch.FloatTensor(batch_seq_num).to(device)]


train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
logger.info('train_loader: {}'.format(len(train_loader)))
logger.info('val_loader: {}'.format(len(val_loader)))
logger.info('test_loader: {}'.format(len(test_loader)))


output_dir = 'ckpt/{}'.format(exp_id)
ensure_dir(output_dir)

if load_init:
    load_dir = 'ckpt/{}'.format(load_exp_id)
    load_path = '{}/{}_{}_{}.pt'.format(load_dir, load_exp_id, model_name, load_epoch)
    logger.info('Load Init model from {}'.format(load_path))
    model.load_state_dict(torch.load(load_path, map_location='cpu'))
    # print(model.device)

2023-05-24 05:39:10,651 - INFO - train_loader: 28174
2023-05-24 05:39:10,653 - INFO - val_loader: 7044
2023-05-24 05:39:10,654 - INFO - test_loader: 3096


In [34]:
for batch_prev_items, batch_locale, batch_len, batch_mask, batch_candidate, \
                batch_label, batch_origin_label, batch_index, batch_seq_cat, batch_seq_num in tqdm(train_loader, desc='train model {}'.format(exp_id), total=len(train_loader)):
    print(batch_prev_items.shape, batch_locale.shape, batch_len.shape, batch_mask.shape, batch_candidate.shape, 
    batch_label.shape, batch_index.shape, batch_origin_label.shape, batch_seq_cat.shape, batch_seq_num.shape)
    break

train model 55974:   0%|          | 0/28174 [00:03<?, ?it/s]

torch.Size([1024, 27]) torch.Size([1024]) torch.Size([1024]) torch.Size([1024, 27]) torch.Size([1024]) torch.Size([1024]) torch.Size([1024]) torch.Size([1024]) torch.Size([1024, 14]) torch.Size([1024, 15])


In [35]:
batch_origin_label

tensor([  75028, 1180767,  282826,  ...,  944979, 1302563,  995582],
       device='cuda:2')

In [36]:
batch_label

tensor([0., 0., 0.,  ..., 0., 0., 1.], device='cuda:2')

In [37]:
batch_candidate    

tensor([1241319, 1306907,  276309,  ..., 1044726, 1015445,  995582],
       device='cuda:2')

In [38]:
batch_index

tensor([3099351, 3173155, 2763189,  ..., 2227456, 2395310, 2521688],
       device='cuda:2')

In [15]:
score, loss = model.predict(batch_prev_items=batch_prev_items, batch_locale=batch_locale, 
                                        batch_candidate_set=batch_candidate, batch_len=batch_len, 
                                        batch_label=batch_label.unsqueeze(-1), batch_mask=batch_mask,
                                        batch_seq_cat=batch_seq_cat, batch_seq_num=batch_seq_num)

In [16]:
loss 

tensor(0.6929, device='cuda:2', grad_fn=<BinaryCrossEntropyBackward0>)

In [70]:
def get_train_label_index(group):
    label = group['train_origin_label'].iloc[0]
    candidate = group['train_candidate'].tolist()
    try:
        index = candidate.index(label)
        rrank = 1 / (index + 1)
    except:
        rrank = 0
    return pd.Series({'label_rrank': rrank})

In [73]:
def get_val_label_index(group):
    label = group['val_origin_label'].iloc[0]
    candidate = group['val_candidate'].tolist()
    try:
        index = candidate.index(label)
        rrank = 1 / (index + 1)
    except:
        rrank = 0
    return pd.Series({'label_rrank': rrank})

In [86]:
def get_test_label_index(group):
    label = group['test_origin_label'].iloc[0]
    candidate = group['test_candidate'].tolist()
    try:
        index = candidate.index(label)
        rrank = 1 / (index + 1)
    except:
        rrank = 0
    return pd.Series({'label_rrank': rrank})

In [71]:
df = pd.DataFrame({
    'train_candidate': train_candidate_list,
    'train_index': train_index_list,
    'train_origin_label': train_origin_label_list,
    'train_y_pred': train_y_pred_list
})
df_sorted = df.sort_values(['train_index', 'train_y_pred'], ascending=[True, False]).reset_index(drop=True)
df_grouped = df_sorted.groupby('train_index').apply(get_train_label_index).reset_index()
train_mrr = np.mean(df_grouped['label_rrank'].values)
train_mrr

0.0786770463862914

In [72]:
log_train_mrr = True

In [84]:
epochs = 1

In [85]:
best_epoch = -1
# 训练
if train:
    logger.info('Training...')
    output_dir = 'ckpt/{}'.format(exp_id)
    ensure_dir(output_dir)
    auc_all = []
    mrr_all = []
    min_val_loss = float('inf')
    max_val_auc = 0.0
    # best_epoch = -1
    for epoch in range(epochs):
        # train
        logger.info('start train epoch {}'.format(epoch))
        model.train()
        train_loss_list = []
        train_candidate_list = []
        train_y_pred_list = []
        train_index_list = []
        train_label_list = []
        train_origin_label_list = []
        for batch_prev_items, batch_locale, batch_len, batch_mask, batch_candidate, \
                batch_label, batch_origin_label, batch_index, batch_seq_cat, batch_seq_num \
                in tqdm(train_loader, desc='train model {}'.format(exp_id), total=len(train_loader)):
            optimizer.zero_grad()
            # y_pred (B, 1), batch_candidate / batch_origin_label / batch_label / batch_index (B, )
            y_pred, loss = model.predict(batch_prev_items=batch_prev_items, batch_locale=batch_locale, 
                                        batch_candidate_set=batch_candidate, batch_len=batch_len, 
                                        batch_label=batch_label.unsqueeze(-1), batch_mask=batch_mask,
                                        batch_seq_cat=batch_seq_cat, batch_seq_num=batch_seq_num)
            # TODO: 去掉？
            loss.backward(retain_graph=True)
            train_loss_list.append(loss.item())
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            with torch.no_grad():
                train_candidate_list += batch_candidate.tolist()
                train_y_pred_list += y_pred.squeeze(-1).tolist()
                train_index_list += batch_index.tolist()
                train_label_list += batch_label.tolist()
                train_origin_label_list += batch_origin_label.tolist()
            break
        # binary-metric
        train_logloss = metrics.log_loss(train_label_list, train_y_pred_list)
        train_auc = metrics.roc_auc_score(train_label_list, train_y_pred_list)
        train_precision = metrics.precision_score(train_label_list, [1 if i >= 0.5 else 0 for i in train_y_pred_list])
        train_recall = metrics.recall_score(train_label_list, [1 if i >= 0.5 else 0 for i in train_y_pred_list])
        train_f1 = metrics.f1_score(train_label_list, [1 if i >= 0.5 else 0 for i in train_y_pred_list])
        logger.info(f"Train Epoch {epoch}, Logloss: {train_logloss:.4f}, AUC: {train_auc:.4f}, \
                    Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1 Score: {train_f1:.4f}")
        # MRR
        if log_train_mrr:
            t1 = time.time()
            df = pd.DataFrame({
                'train_candidate': train_candidate_list,
                'train_index': train_index_list,
                'train_origin_label': train_origin_label_list,
                'train_y_pred': train_y_pred_list
            })
            df_sorted = df.sort_values(['train_index', 'train_y_pred'], ascending=[True, False]).reset_index(drop=True)
            df_grouped = df_sorted.groupby('train_index').apply(get_train_label_index)
            train_mrr = np.mean(df_grouped['label_rrank'].values)
            t2 = time.time()
            logger.info(f"Train Epoch {epoch}, MRR: {train_mrr:.4f}, Time: {t2-t1:.4f}")
        else:
            train_mrr = 0
        train_loss = np.mean(train_loss_list)
        # val
        val_loss_list= []
        val_candidate_list = []
        val_y_pred_list = []
        val_index_list = []
        val_label_list = []
        val_origin_label_list = []
        with torch.no_grad():
            model.eval()
            for batch_prev_items, batch_locale, batch_len, batch_mask, batch_candidate, \
                batch_label, batch_origin_label, batch_index, batch_seq_cat, batch_seq_num \
                in tqdm(val_loader, desc='val model {}'.format(exp_id), total=len(val_loader)):
                # y_pred (B, 1), batch_candidate / batch_origin_label / batch_label / batch_index (B, )
                y_pred, loss = model.predict(batch_prev_items=batch_prev_items, batch_locale=batch_locale, 
                                            batch_candidate_set=batch_candidate, batch_len=batch_len, 
                                            batch_label=batch_label.unsqueeze(-1), batch_mask=batch_mask,
                                            batch_seq_cat=batch_seq_cat, batch_seq_num=batch_seq_num)
                val_loss_list.append(loss.item())
                with torch.no_grad():
                    val_candidate_list += batch_candidate.tolist()
                    val_y_pred_list += y_pred.squeeze(-1).tolist()
                    val_index_list += batch_index.tolist()
                    val_label_list += batch_label.tolist()
                    val_origin_label_list += batch_origin_label.tolist()
                break
        # binary-metric
        val_logloss = metrics.log_loss(val_label_list, val_y_pred_list)
        val_auc = metrics.roc_auc_score(val_label_list, val_y_pred_list)
        val_precision = metrics.precision_score(val_label_list, [1 if i >= 0.5 else 0 for i in val_y_pred_list])
        val_recall = metrics.recall_score(val_label_list, [1 if i >= 0.5 else 0 for i in val_y_pred_list])
        val_f1 = metrics.f1_score(val_label_list, [1 if i >= 0.5 else 0 for i in val_y_pred_list])
        logger.info(f"Val Epoch {epoch}, Logloss: {val_logloss:.4f}, AUC: {val_auc:.4f}, \
                    Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1 Score: {val_f1:.4f}")
        # MRR
        if log_train_mrr:
            t1 = time.time()
            df = pd.DataFrame({
                'val_candidate': val_candidate_list,
                'val_index': val_index_list,
                'val_origin_label': val_origin_label_list,
                'val_y_pred': val_y_pred_list
            })
            df_sorted = df.sort_values(['val_index', 'val_y_pred'], ascending=[True, False]).reset_index(drop=True)
            df_grouped = df_sorted.groupby('val_index').apply(get_val_label_index)
            val_mrr = np.mean(df_grouped['label_rrank'].values)
            t2 = time.time()
            logger.info(f"Val Epoch {epoch}, MRR: {train_mrr:.4f}, Time: {t2-t1:.4f}")
        else:
            val_mrr = 0
        val_loss = np.mean(val_loss_list)
        
        mrr_all.append(val_mrr)
        auc_all.append(val_auc)
        lr_scheduler.step(val_auc)
        lr = optimizer.param_groups[0]['lr']
        if log_train_mrr:
            logger.info('Epoch {}/{} Complete: Train Loss {:.6f}, Train MRR {:.6f}, Val Loss {:.6f}, Val MRR {:.6f}, lr {}'.format(
                epoch, epochs, train_loss, train_mrr, val_loss, val_mrr, lr))
        else:
            logger.info('Epoch {}/{} Complete: Train Loss {:.6f}, Val Loss {:.6f}, lr {}'.format(epoch, epochs, train_loss, val_loss, lr))
        if val_auc > max_val_auc:
            min_val_loss = val_loss
            max_val_auc = val_auc
            best_epoch = epoch
            save_path = '{}/{}_{}.pt'.format(output_dir, exp_id, model_name)
            logger.info('Save model to {}'.format(save_path))
            torch.save(model.state_dict(), save_path)
        if lr < early_stop_lr:
            logger.info('early stop')
            break

    # load best epoch
    assert best_epoch == np.argmax(auc_all)
    logger.info('best_epoch {}, AUC = {}'.format(best_epoch, max_val_auc))
    load_path = '{}/{}_{}.pt'.format(output_dir, exp_id, model_name)
    logger.info('Load model from {}'.format(load_path))
    model.load_state_dict(torch.load(load_path))
    save_path = '{}/{}_{}_{}.pt'.format(output_dir, exp_id, model_name, best_epoch)
    logger.info('Save model to {}'.format(save_path))
    torch.save(model.state_dict(), save_path)

2023-05-24 06:46:53,616 - INFO - Training...
2023-05-24 06:46:53,618 - INFO - start train epoch 0


train model 55974:   0%|          | 0/28174 [00:03<?, ?it/s]

2023-05-24 06:46:57,315 - INFO - Train Epoch 0, Logloss: 0.2517, AUC: 0.6519,                     Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


2023-05-24 06:46:57,527 - INFO - Train Epoch 0, MRR: 0.0752, Time: 0.2114


val model 55974:   0%|          | 0/7044 [00:00<?, ?it/s]

2023-05-24 06:46:58,109 - INFO - Val Epoch 0, Logloss: 0.2878, AUC: 0.6492,                     Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


2023-05-24 06:46:58,329 - INFO - Train Epoch 0, MRR: 0.0752, Time: 0.2190
2023-05-24 06:46:58,330 - INFO - Epoch 0/1 Complete: Train Loss 0.251707, Train MRR 0.075195, Val Loss 0.287762, Val MRR 0.089844, lr 0.001
2023-05-24 06:46:58,330 - INFO - Save model to ckpt/55974/55974_MatchModelV2withATTMatchFold0.pt
2023-05-24 06:47:15,091 - INFO - best_epoch 0, AUC = 0.6491532935249114
2023-05-24 06:47:15,092 - INFO - Load model from ckpt/55974/55974_MatchModelV2withATTMatchFold0.pt
2023-05-24 06:47:21,393 - INFO - Save model to ckpt/55974/55974_MatchModelV2withATTMatchFold0_0.pt


In [110]:
# 开始评估
logger.info('Testing...')
test_candidate_list = []
test_y_pred_list = []
test_index_list = []
test_label_list = []
test_origin_label_list = []
model.eval()

for batch_prev_items, batch_locale, batch_len, batch_mask, batch_candidate, \
    batch_label, batch_origin_label, batch_index, batch_seq_cat, batch_seq_num \
        in tqdm(test_loader, desc='test model {}'.format(exp_id), total=len(test_loader)):
    # y_pred (B, 1), batch_candidate / batch_origin_label / batch_label / batch_index (B, )
    y_pred, loss = model.predict(batch_prev_items=batch_prev_items, batch_locale=batch_locale, 
                                batch_candidate_set=batch_candidate, batch_len=batch_len, 
                                batch_label=batch_label.unsqueeze(-1), batch_mask=batch_mask,
                                batch_seq_cat=batch_seq_cat, batch_seq_num=batch_seq_num)
    with torch.no_grad():
        test_candidate_list += batch_candidate.tolist()
        test_y_pred_list += y_pred.squeeze(-1).tolist()
        test_index_list += batch_index.tolist()
        test_label_list += batch_label.tolist()
        test_origin_label_list += batch_origin_label.tolist()
        break

# # binary-metric
# test_logloss = metrics.log_loss(test_label_list, test_y_pred_list)
# test_auc = metrics.roc_auc_score(test_label_list, test_y_pred_list)
# test_precision = metrics.precision_score(test_label_list, [1 if i >= 0.5 else 0 for i in test_y_pred_list])
# test_recall = metrics.recall_score(test_label_list, [1 if i >= 0.5 else 0 for i in test_y_pred_list])
# test_f1 = metrics.f1_score(test_label_list, [1 if i >= 0.5 else 0 for i in test_y_pred_list])
# logger.info(f"Val Epoch {epoch}, Logloss: {test_logloss:.4f}, AUC: {test_auc:.4f}, \
#             Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}")
# MRR
t1 = time.time()
df = pd.DataFrame({
    'test_candidate': test_candidate_list,
    'test_index': test_index_list,
    'test_origin_label': test_origin_label_list,
    'test_y_pred': test_y_pred_list
})
df_sorted = df.sort_values(['test_index', 'test_y_pred'], ascending=[True, False]).reset_index(drop=True)
df_grouped = df_sorted.groupby('test_index').apply(get_test_label_index)
df_test_preds = df_sorted.groupby('test_index').apply(lambda group : group['test_candidate'].tolist())
test_mrr = np.mean(df_grouped['label_rrank'].values)
t2 = time.time()
logger.info(f"Test Epoch {epoch}, MRR: {train_mrr:.4f}, Time: {t2-t1:.4f}")

2023-05-24 07:00:44,001 - INFO - Testing...


test model 55974:   0%|          | 0/3096 [00:00<?, ?it/s]

2023-05-24 07:00:44,061 - INFO - Test Epoch 0, MRR: 0.0752, Time: 0.0301


In [106]:
logger.info('Decoding the results...')
test_res_unencoded = []
for ind, x in tqdm(enumerate(range(len(test_preds_encoded))), total=len(test_preds_encoded)):
    x = df_test_preds.iloc[ind] + test_preds_encoded[ind][len_candidate_set:]
    assert len(x) == 100
    x_unencoded = [id2product[id_] for id_ in x]
    test_res_unencoded.append(x_unencoded)

  0%|          | 102/316972 [00:00<00:52, 6055.90it/s]


AssertionError: 

In [ ]:
logger.info('Saving...')
df_test['next_item_prediction'] = test_res_unencoded
logger.info(df_test['next_item_prediction'].apply(len).describe())
logger.info('Save Fold {} res to {}'.format(Fold, 'output/{}_{}_{}_{}.parquet'.format(seed, exp_id, model_name, best_epoch)))
df_test[['locale', 'next_item_prediction']].to_parquet('output/{}_{}_{}_{}.parquet'.format(seed, exp_id, model_name, best_epoch), engine='pyarrow')
logger.info('Save Fold {} score to {}, shape = {}'.format(Fold, 'output/{}_{}_{}_{}.npy'.format(seed, exp_id, model_name, best_epoch), test_scores.shape))
np.save('output/scores_{}_{}_{}_{}_Fold{}.npy'.format(seed, exp_id, model_name, best_epoch, Fold), test_scores)
np.save('output/preds_{}_{}_{}_{}_Fold{}.npy'.format(seed, exp_id, model_name, best_epoch, Fold), test_res)